<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/cohere-developer-experience/blob/main/notebooks/Data_Analyst_Agent_Cohere_and_Langchain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Tool Use - Simple Data Analyst Agent with Cohere and Langchain

Tool use is a method whichs allows developers to connect Cohere's Command models to external tools like search engines, APIs, databases, and other software tools. Just like how [Retrieval-Augmented Generation (RAG)](https://docs.cohere.com/docs/retrieval-augmented-generation-rag) allows a model to use an external data source to improve factual generation, tool use is a capability that allows retrieving data from multiple sources. But it goes beyond simply retrieving information and is able to use software tools to execute code, or even create entries in a CRM system.

In this notebook, we'll see how we can use two tools to create a simple data analyst agent that is able to search the web and run code in a python interpreter. This agent uses Cohere's Command R+ mode and Langchain.


![](https://raw.githubusercontent.com/cohere-ai/cohere-developer-experience/main/notebooks/images/tool-use-notebook-header.png)

# Environment
Let's start by installing the required libraries

In [21]:
! pip install --quiet langchain langchain_cohere langchain_experimental


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Setup
We'll need a Cohere API key here. Grab your key and paste it in the next slide if you have one, or [register](https://dashboard.cohere.ai/welcome/register) and create a new API key.

In [22]:
### LLMs
import os
#os.environ['COHERE_API_KEY'] = "<INSERT COHERE API KEY HERE>"
os.environ['COHERE_API_KEY'] = "Kfz4IJ1w16XXtiB2eTUQGv5uxxs0J5rwhE96PeHP"

In [23]:
# Create the Cohere chat model
from langchain_cohere.chat_models import ChatCohere
chat = ChatCohere(model="command-a-03-2025", temperature=0.3)

# Define tools
Our simple data analyst will be equipped with a web search tool, and a python interpreter (which we can use to run plotting code, for example).

### Web search
Let's first equip our agent with web search! We can use the Tivaly API for this. Head on to [tavily.com](https://tavily.com) and grab an API key to use here.

In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults

#os.environ['TAVILY_API_KEY'] = "<INSERT TAVILY API KEY HERE>"
os.environ['TAVILY_API_KEY'] = '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'

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


from pydantic import BaseModel, Field
class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

### Python interpreter tool
Let's equip our agent with a python interpreter!

In [25]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
repl_tool.args_schema = ToolInput


# Create ReAct Agent: the data analyst

In [26]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [27]:
# Create the prompt
prompt = ChatPromptTemplate.from_template("{input}")


# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[internet_search, repl_tool],
    prompt=prompt,
)

In [28]:
agent_executor = AgentExecutor(agent=agent,
                               tools=[internet_search, repl_tool],
                               verbose=True)

Let's ask the data analyst a question

In [29]:
agent_executor.invoke({
    "input": "Create a plot of the number of full time employees at the 3 tech companies with the highest market cap in the United States in 2024.",
})



> Entering new AgentExecutor chain...


ERROR:root:Failed to parse multihop completion for input: <|START_RESPONSE|>Plan: I will first search for the three tech companies with the highest market cap in the United States in 2024. Then, I will search for the number of full-time employees at each of these companies. Finally, I will use Python to create a plot of the number of full-time employees at these companies.

Action: 
```json
{
  "tool_name": "internet_search",
  "parameters": {
    "query": "top 3 tech companies by market cap in the United States 2024"
  }
}
```<|END_RESPONSE|>
ERROR:root:Error: Failed to parse multihop completion for input: <|START_RESPONSE|>Plan: I will first search for the three tech companies with the highest market cap in the United States in 2024. Then, I will search for the number of full-time employees at each of these companies. Finally, I will use Python to create a plot of the number of full-time employees at these companies.

Action: 
```json
{
  "tool_name": "internet_search",
  "parameters

ValueError: Action Parsing Failed: <|START_RESPONSE|>Plan: I will first search for the three tech companies with the highest market cap in the United States in 2024. Then, I will search for the number of full-time employees at each of these companies. Finally, I will use Python to create a plot of the number of full-time employees at these companies.

Action: 
```json
{
  "tool_name": "internet_search",
  "parameters": {
    "query": "top 3 tech companies by market cap in the United States 2024"
  }
}
```<|END_RESPONSE|>

In [10]:
# notice that the model did websearch calls, then used python to do some analysis
# the model can even self-debug itself if the python code was wrong

# Let's ask another question to the data analyst -- which requires a direct answer

In [11]:
agent_executor.invoke({
    "input": "Hey how are you?",
})



> Entering new AgentExecutor chain...

I will respond to the user's greeting.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [internet_search, python_interpreter].Relevant Documents: None
Cited Documents: None
Answer: Hello! I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?
Grounded answer: Hello! I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?

> Finished chain.


{'input': 'Hey how are you?',
 'output': "Hello! I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?",
 'citations': []}

This is a simple example to get you start. There [are many tools](https://python.langchain.com/docs/integrations/tools/) you can equip your agent with. Once youre comfortable with these ideas, you can also proceed to define your tools (see [Multi-step tool use in Action](https://docs.cohere.com/docs/multi-step-tool-use#multi-step-tool-use-in-action)).
